In [1]:
import numpy as np

In [2]:
import os

In [3]:
before_folders = os.listdir('before')
after_folders = os.listdir('after')
step = 8
before_folders.sort()
after_folders.sort()
print(before_folders)
print(after_folders)

['shake_before', 'sin_before', 'squeeze_before', 'stab_before', 'swing_before']
['shake_after', 'sin_after', 'squeeze_after', 'stab_after', 'swing_after']


In [3]:
befores = []
afters = []
step = 8
for folder in before_folders:
    buffer = []
    folder_file = os.listdir('before/'+folder)
    for file in folder_file:
        if file.endswith('state.npz'):
            buffer.append(file)
    buffer.sort(key=lambda x:int(x[:-10]))
    for i in range(len(buffer)):
        buffer[i] = folder + '/' + buffer[i]
    befores += buffer

for folder in after_folders:
    buffer = []
    folder_file = os.listdir('after/'+folder)
    for file in folder_file:
        if file.endswith('state.npz'):
            buffer.append(file)
    buffer.sort(key=lambda x:int(x[:-10]))
    for i in range(len(buffer)):
        buffer[i] = folder + '/' + buffer[i]
    afters += buffer
print(befores)

NameError: name 'before_folders' is not defined

In [3]:
before_files = os.listdir('before')
after_files = os.listdir('after')
step=8
afters = []
befores = []
for before in before_files:
    if before.endswith("state.npz"):
        befores.append(before)

for after in after_files:
    if after.endswith('state.npz'):
        afters.append(after)
befores.sort(key=lambda x:int(x[:-10]))
afters.sort(key=lambda x:int(x[:-10]))

In [4]:
#actions = np.vstack([np.load('action_shake.npy'),np.load('action_sin.npy'),np.load('action_squeeze.npy'),
#                     np.load('action_stab.npy'),np.load('action_swing.npy')])
actions = np.load('action.npy')
print(actions.shape)

(10000, 6)


In [7]:
size = 10000 - 200*(step-2)
before_x = np.zeros((size,8192,3),dtype=np.float64)
before_v = np.zeros((size,8192,3),dtype=np.float64)
before_F = np.zeros((size,8192,3,3),dtype=np.float64)
before_C = np.zeros((size,8192,3,3),dtype=np.float64)
before_p = np.zeros((size,21), dtype=np.float64)
after_x =  np.zeros((size,step-1,8192,3),dtype=np.float64)
after_p = np.zeros((size,step-1,21),dtype=np.float64)
action_ =  np.zeros((size,step-1,actions.shape[1]),dtype=np.float64)

In [8]:
cnt = 0
for i in range(10000):
    if i % 50 <= (50-(step-1)):
        z = np.load('before/'+befores[i])
        before_x[cnt] = z['arr_0']
        before_v[cnt] = z['arr_1']
        before_F[cnt] = z['arr_2']
        before_C[cnt] = z['arr_3']
        before_p[cnt] = np.hstack([z['arr_4'],z['arr_5'],z['arr_6']])
        checker = []
        for j in range(step-1):
            #checker.append(afters[i+j][:2])
            z = np.load('after/'+afters[i+j])
            after_x[cnt,j] = z['arr_0']
            after_p[cnt,j] = np.hstack([z['arr_4'],z['arr_5'],z['arr_6']])
            action_[cnt,j] = actions[i+j]
        #assert(max(checker)==min(checker))
        cnt += 1
actions = action_

In [9]:
size

8800

In [10]:
sort_index = np.random.choice(size,size,replace=False)
small_index = np.random.choice(size,1000,replace=False)

In [11]:
np.savez(
    'data/rope.npz',
    before_x = before_x[sort_index],
    before_v = before_v[sort_index],
    before_F = before_F[sort_index],
    before_C = before_C[sort_index],
    before_p = before_p[sort_index],
    after_x = after_x[sort_index],
    action = actions[sort_index]
)
np.savez(
    'data/rope_small.npz',
    before_x = before_x[small_index],
    before_v = before_v[small_index],
    before_F = before_F[small_index],
    before_C = before_C[small_index],
    before_p = before_p[small_index],
    after_x = after_x[small_index],
    action = actions[small_index]
)

In [14]:
z = np.load('data/chopsticks.npz')

In [15]:
z['after_x'].shape

(8800, 7, 8192, 3)

In [16]:
z['before_C'].shape

(8800, 8192, 3, 3)